This is solution for Lesson 1 of Fast.ai course on DL.  Solution is using Keras 2, and re-uses ideas from great "Deep Learning with Python" book by Keras's author.

In [1]:
%matplotlib inline

In [2]:
# Importing base functions
from __future__ import division,print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

Import Keras's functions

In [3]:
from keras.applications import VGG16
from keras import backend as K
from keras.regularizers import l2, l1
from keras.preprocessing import image
from keras import models
from keras import layers
from keras import optimizers
from keras import callbacks
#from keras.utils.training_utils import multi_gpu_model # only in 2.0.9

Using TensorFlow backend.


Loading VGG 16 model from Keras itself

In [4]:
input_dim = 224
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(input_dim, input_dim, 3))

In [5]:
base_dir = "/home/ott/work/datasets/dogs-vs-cats-redux-kernels-edition/"

train_dir = os.path.join(base_dir, 'train')
valid_dir = os.path.join(base_dir, 'valid')
test_dir = os.path.join(base_dir, 'test')

batch_size = 20

In [6]:
train_datagen = image.ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size = (input_dim, input_dim),
        batch_size = batch_size,
        class_mode='binary',
        shuffle=True,
        classes=['cats','dogs'])

valid_generator = image.ImageDataGenerator(rescale=1./255).flow_from_directory(
        valid_dir,
        target_size=(input_dim, input_dim),
        batch_size = batch_size,
        class_mode='binary',
        shuffle=False,
        classes=['cats','dogs'])

test_generator = image.ImageDataGenerator(rescale=1./255).flow_from_directory(
        test_dir,
        target_size=(input_dim, input_dim),
        batch_size = batch_size,
        class_mode='binary',
        shuffle=False,
        classes=['cats','dogs'])

Found 21000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [7]:
# This model is disables most of the layers of VGG16, and enabling only the last layers for retraining.
vgg16.trainable = True

for layer in vgg16.layers:
    layer.trainable = layer.name.startswith('block5_')

model = models.Sequential()
model.add(vgg16)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               6422784   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 21,137,729
Trainable params: 13,502,465
Non-trainable params: 7,635,264
_________________________________________________________________


In [8]:
best_model_name = '/home/ott/work/DL/dogs_cats.h5'
callbacks_list = [callbacks.EarlyStopping(monitor='acc', patience=2),
                  callbacks.ModelCheckpoint(filepath=best_model_name,
                                monitor='val_loss', save_best_only=True)
                 ]
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size,
      epochs=20,
      callbacks=callbacks_list,
      validation_data=valid_generator,
      validation_steps=valid_generator.samples/valid_generator.batch_size)

Epoch 1/20
1050/1050 [==============================] - 191s - loss: 0.2544 - acc: 0.8911 - val_loss: 0.1117 - val_acc: 0.9590
Epoch 2/20
1050/1050 [==============================] - 181s - loss: 0.1572 - acc: 0.9361 - val_loss: 0.0947 - val_acc: 0.9655
Epoch 3/20
1050/1050 [==============================] - 177s - loss: 0.1401 - acc: 0.9464 - val_loss: 0.1363 - val_acc: 0.9595
Epoch 4/20
1050/1050 [==============================] - 178s - loss: 0.1339 - acc: 0.9495 - val_loss: 0.0735 - val_acc: 0.9735
Epoch 5/20
1050/1050 [==============================] - 179s - loss: 0.1271 - acc: 0.9515 - val_loss: 0.0704 - val_acc: 0.9765
Epoch 6/20
1050/1050 [==============================] - 177s - loss: 0.1218 - acc: 0.9529 - val_loss: 0.1019 - val_acc: 0.9635
Epoch 7/20
1050/1050 [==============================] - 177s - loss: 0.1249 - acc: 0.9547 - val_loss: 0.0834 - val_acc: 0.9740
Epoch 8/20
1050/1050 [==============================] - 177s - loss: 0.1263 - acc: 0.9550 - val_loss: 0.0835 - 

In [9]:
# evaluate on model itself
print(model.evaluate_generator(test_generator, test_generator.samples/test_generator.batch_size))

# Load the best model & evaluate on it...
best_model = models.load_model(best_model_name)
test_res = best_model.evaluate_generator(test_generator, test_generator.samples/test_generator.batch_size)
print(test_res)


[0.10951651223760564, 0.96799999535083769]
[0.08541554275521776, 0.96849999368190764]


In [10]:
kaggle_generator = image.ImageDataGenerator(rescale=1./255).flow_from_directory(
        base_dir + 'test-orig',
        target_size=(input_dim, input_dim),
        batch_size = batch_size,
        shuffle=False,
        class_mode=None)

kaggle_results = best_model.predict_generator(kaggle_generator, kaggle_generator.samples/kaggle_generator.batch_size)
kaggle_filenames = kaggle_generator.filenames

Found 12500 images belonging to 1 classes.


In [11]:
print(train_generator.class_indices)
pairs = list(zip(kaggle_filenames, kaggle_results))
#print(pairs[0:20])


{'dogs': 1, 'cats': 0}


In [12]:
sorted_results = np.zeros(len(kaggle_results))
for i in range(0, len(kaggle_results)):
    file_idx = int(os.path.splitext(os.path.basename(kaggle_filenames[i]))[0])-1
    sorted_results[file_idx] = kaggle_results[i][0]

sorted_results = np.clip(sorted_results, 0.05, 0.95)
sorted_results[0:10]

array([ 0.95,  0.95,  0.95,  0.95,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05])

In [13]:
with open('kaggle_submit.csv', 'w') as f:
    f.write('id,label\n')
    for i in range(0, len(sorted_results)):
        f.write(str(i+1)+','+str(sorted_results[i])+'\n')
        
# could also use following code if it could be represented as matrix...
# np.savetxt('submission.csv', subm, fmt='%d,%.5f', header='id,label', comments='')